<a href="https://colab.research.google.com/github/makeflower99/test/blob/master/%EA%B0%80%EC%84%A41_%EC%9D%B8%ED%94%84%EB%9F%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
from collections import Counter
from konlpy.tag import Hannanum
import re
import pandas as pd
import time
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

#크롤링한 결과 불러오기
# 'Inflearn.csv'파일과 '불용어리스트.csv'는 파일위치에서 최상위부분에 위치하여야한다.
#파일 넣는 방법 좌측에 폴더를 클릭한 후
# ..(상위폴더로 이동)을 선택한 후 파일 업로드 버튼을 눌러 업로드 한다.
# 저장된 파일은 content폴더에 위치해있습니다.

data_o = pd.read_csv("/Inflearn.csv")
data = data_o.copy()
#  KoNLPy 사용
# 사용법 확인하기
def remove_special_characters(df):
    for title in df.title:
        df['title'] = df['title'].str.replace(r"[^\w\s]", " ", regex=True)
    return df

data = remove_special_characters(data)

# 문장에서 명사 추출
hannanum = Hannanum()
nouns = []
for text in data['title']:
    nouns.extend(hannanum.nouns(text))
count = Counter(nouns)
#명사 빈도 추출 - 상위 단어 20개 추출
noun_list = count.most_common(20)
#불용어 처리 - 불용어 제거 리스트?
#숫자까지 제거 데이터 프레임 : df_done1
#명사만 : nouns
korean_stopwords_path = "/불용어리스트.csv"

# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path, encoding='utf-8') as f:
	stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
meaningful_words = [w for w in nouns if not w in stopwords]
count1 = Counter(meaningful_words)
#명사 빈도 추출 - 상위 단어 100개 추출
noun_list = count1.most_common(20)
# 빈 데이터 프레임 생성
df_fin = pd.DataFrame(columns=['word', 'count'])

# frequent2 리스트에서 데이터 프레임에 데이터 추가
for word, count in noun_list:
    df_fin.loc[len(df_fin)] = [word, count]

df_fin.to_csv("Inflearn_ko.csv",index=False)

data_en = data_o.copy()

def remove_non_english(df):
    df['title'] = df['title'].str.replace(r"[^a-zA-Z\s]", "", regex=True)
    return df
data_en = remove_non_english(data_en)

def remove_non_english(df):
    df['title'] = df['title'].str.replace(r"[^a-zA-Z\s]", "", regex=True)
    return df
data_en = remove_non_english(data_en)
compile = re.compile("\W+")
for i in range(len(data_en)):
  a = compile.sub(" ", data_en["title"][i])  # "text" 열을 수정한다고 가정
  data_en.loc[i, "title"] = a.lower()  # "text" 열의 값을 수정
#불용어 제거
#불용어 데이터 다운로드
nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_word_eng = set(stopwords.words('english'))
line2 = [i for i in data_en['title'] if i not in stop_word_eng]
#어간 추출 대신 표제어 추출(lemmatization)을 사용


lemmatizer = WordNetLemmatizer()
token = RegexpTokenizer('[\w]+')
result_pre_lem = [token.tokenize(i) for i in line2]
middle_pre_lem= [r for i in result_pre_lem for r in i]
final_lem = [lemmatizer.lemmatize(i) for i in middle_pre_lem if not i in stop_word_eng] # 불용어 제거

#표제어 빈도수
english2 = pd.Series(final_lem).value_counts().head(20)

# 표제어
english2 = pd.Series(final_lem).value_counts().head(22)

# 데이터 프레임으로 변환
df = english2.reset_index(name="count")

# 열 이름 변경
df.columns = ["word", "count"]

# 결과 출력
df = df[~df["word"].isin(["part", "j"])]

df.to_csv("Inflearn_en.csv",index=False)